In [10]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier

# مسیر فایل‌ها
DATA_DIR = '../data'
FEATURE_COUNT = 4125

# ------------------------------------
# تبدیل داده‌های LibSVM به DataFrame
# ------------------------------------
def parse_libsvm_line(line):
    parts = line.strip().split()
    label = int(parts[0].replace('+', ''))
    features = [0.0] * FEATURE_COUNT
    for item in parts[1:]:
        index, value = item.split(":")
        index = int(index)
        if 1 <= index <= FEATURE_COUNT:
            features[index - 1] = float(value)
    return [label] + features

def load_all_data():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".txt"):
            filepath = os.path.join(DATA_DIR, filename)
            print(f"📄 Reading {filename} ...")
            with open(filepath, "r") as file:
                for line in tqdm(file, desc=filename):
                    row = parse_libsvm_line(line)
                    data.append(row)
    columns = ['label'] + [f'feature_{i}' for i in range(1, FEATURE_COUNT + 1)]
    df = pd.DataFrame(data, columns=columns)
    return df

# ------------------------------------
# حذف ویژگی‌های با مقدار تکراری
# ------------------------------------
def remove_single_value_columns(df):
    cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
    print(f"🧹 حذف {len(cols_to_drop)} ویژگی با مقدار تکراری")
    return df.drop(columns=cols_to_drop)

# ------------------------------------
# نرمال‌سازی داده‌ها
# ------------------------------------
def normalize(df):
    X = df.drop(columns=['label'])
    y = df['label']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
    return pd.concat([y.reset_index(drop=True), X_scaled_df], axis=1)

# ------------------------------------
# حذف داده‌های پرت
# ------------------------------------
def replace_outliers_with_mean(df):
    labels = df['label']
    features = df.drop(columns=['label'])
    cleaned = features.copy()
    for col in cleaned.columns:
        Q1 = cleaned[col].quantile(0.25)
        Q3 = cleaned[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = cleaned[(cleaned[col] < lower) | (cleaned[col] > upper)][col]
        if len(outliers) > 0:
            if len(outliers) > 1000:
                distances = np.maximum(outliers - upper, lower - outliers)
                top_outliers = distances.abs().sort_values(ascending=False).head(1000).index
            else:
                top_outliers = outliers.index
            mean_val = cleaned[(cleaned[col] >= lower) & (cleaned[col] <= upper)][col].mean()
            cleaned.loc[top_outliers, col] = mean_val
    cleaned['label'] = labels
    return cleaned

# ------------------------------------
# انتخاب ویژگی‌های مهم با XGBoost
# ------------------------------------
def select_important_features(X, y, threshold="mean"):
    try:
        y = np.array(y)
        y = np.where(y == -1, 0, y)
        if isinstance(X, pd.DataFrame):
            feature_names = X.columns
            X = X.select_dtypes(include=[np.number])
            X_array = X.values
        else:
            X_array = X
            feature_names = [f'feature_{i}' for i in range(X_array.shape[1])]
        model = XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='logloss')
        model.fit(X_array, y)
        selector = SelectFromModel(model, threshold=threshold, prefit=True)
        X_selected_array = selector.transform(X_array)
        if X_selected_array.shape[1] == 0:
            print("⚠️ هیچ ویژگی‌ای انتخاب نشد. بازگشت به ورودی اولیه.")
            return pd.DataFrame(X_array, columns=feature_names), y
        selected_cols = [col for col, keep in zip(feature_names, selector.get_support()) if keep]
        X_selected_df = pd.DataFrame(X_selected_array, columns=selected_cols)
        print(f"✅ انتخاب {len(selected_cols)} ویژگی مهم (براساس threshold = {threshold})")
        return X_selected_df, y
    except Exception as e:
        print(f"❌ خطا در انتخاب ویژگی‌ها: {e}")
        return pd.DataFrame(X_array, columns=feature_names), y

# ------------------------------------
# تولید ویژگی‌های چند جمله‌ای
# ------------------------------------
def add_polynomial_features(X, degree=2):
    try:
        top_features = X.iloc[:, :10]
        poly = PolynomialFeatures(degree=degree, interaction_only=True, include_bias=False)
        X_poly = poly.fit_transform(top_features)
        poly_feature_names = poly.get_feature_names_out(top_features.columns)
        df_poly = pd.DataFrame(X_poly, columns=poly_feature_names, index=X.index)
        print(f"➕ ویژگی‌های چندجمله‌ای اضافه شد: {df_poly.shape[1]}")
        return pd.concat([X, df_poly], axis=1)
    except Exception as e:
        print(f"❌ خطا در تولید ویژگی‌های چندجمله‌ای: {e}")
        return X

# ------------------------------------
# اجرای کامل پیش‌پردازش
# ------------------------------------
def full_preprocessing_pipeline():
    print("🚀 شروع پیش‌پردازش...")
    df = load_all_data()
    df = remove_single_value_columns(df)
    df = normalize(df)
    df = replace_outliers_with_mean(df)

    X = df.drop(columns=['label'])
    y = df['label']

    # انتخاب ویژگی مهم اولیه
    X_selected, y = select_important_features(X, y, threshold="mean")

    # افزودن ویژگی‌های چند جمله‌ای
    X_extended = add_polynomial_features(X_selected)

    # انتخاب مجدد ویژگی‌های مهم
    X_final, y = select_important_features(X_extended, y, threshold="mean")

    final_df = pd.concat([X_final, pd.Series(y, name='label')], axis=1)
    print("✅ پیش‌پردازش کامل شد.")
    print(f"📊 شکل نهایی داده: {final_df.shape}")
    return final_df

# ------------------------------------
# اجرای برنامه
# ------------------------------------
if __name__ == '__main__':
    final_df = full_preprocessing_pipeline()
final_df.to_csv("../data/FinalPreProcessing.csv", index=False)
print("📁 فایل نهایی ذخیره شد: FinalPreProcessing.csv")

🚀 شروع پیش‌پردازش...
📄 Reading BBC.txt ...


BBC.txt: 0it [00:00, ?it/s]

BBC.txt: 17720it [00:03, 4843.64it/s]


📄 Reading CNN.txt ...


CNN.txt: 22545it [00:05, 3984.38it/s]


...

🧹 حذف 3894 ویژگی با مقدار تکراری


c:\Users\niman\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:58:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ انتخاب 43 ویژگی مهم (براساس threshold = mean)
➕ ویژگی‌های چندجمله‌ای اضافه شد: 55


c:\Users\niman\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:58:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ انتخاب 26 ویژگی مهم (براساس threshold = mean)
✅ پیش‌پردازش کامل شد.
📊 شکل نهایی داده: (129685, 27)
📁 فایل نهایی ذخیره شد: FinalPreProcessing.csv
